# Deployment
> run this notebook during deployment

To check time offset correction use [https://www.utctime.net](https://www.utctime.net)

> Data from the RTE website collected from 0300 - 0400 Z is averaged and attributed to the time 0300Z. So when you submit a forecast between 0350-0410 Z , it is really meant for a T+0 = 0300Z. Finally, 0300Z + 18 hours = 2100Z as given in the response.

USE `Energy[t-1, t-2,...]`, and `Forecast[t+17]` , i.e. `Energy[t-1]` is current reading!

our password is` 961993551 `
```python
Y_t18_prediction = 3500
url_to_submit = f'http://3.1.52.222/submit/pred?pwd=961993551&value={Y_t18_prediction}'

with urllib.request.urlopen(url_to_submit) as response:
   html_valid = response.read()
```
then convert to pyhton str
```python
str(html_valid.decode())
```
if valid submission, we will receive:
```
> 'Submitted \nReceived: 20-Jul-2020 04:02:50 UTC \nFor     : 20-Jul-2020 21:00:00 UTC \nValue   : 3500 \n'
```
and for wrong timing/passwords:
```
if password is wrong:
> 'Password invalid \n'
if submission is too early or late:
> 'Submission time closed \n'
```

an example for wrong submission code
```python
html=b''
try:
    with urllib.request.urlopen('http://3.1.52.222/submit/pred?pwd=961993551&value=3500') as response:
        html = response.read()
except urllib.request.HTTPError:
    html = b'server down'

print(html.decode())
```

`> server down`<br>
or<br>
```python
html.decode().split('\n')
```
`> ['Password invalid ', '']`

# Deploy Models

In [142]:
# # Set to FALSE when deploying!!! # #
TESTING = False

# # # # # # # # # # # # # # # # # # #
import os, glob, pickle
import urllib.request
import torch, numpy as np

import os, pandas as pd, numpy as np
import time, datetime

if torch.cuda.is_available():
#     device = torch.device('cuda')
    device = torch.device('cuda:3')
else:
    device = torch.device("cpu")
print(f'Available device: {device}')


#For running in JupyterHub:
if os.path.basename(os.getcwd())!='P003':
    print('Not in /P003 folder, changing directory to P003')
    lib_path = os.path.expanduser(os.path.relpath('~/images/codesDIR/datathon2020/P003'))
    os.chdir(lib_path)

# Import src.datautils
    
from src import datautils

# import matplotlib
#plt.style.use('ggplot')
#%matplotlib inline

# # # # # # # # # # # # # # # # # # #
#             CONSTANTS             #
# # # # # # # # # # # # # # # # # # #
window_size = 40
lead_time = 18

# Normalisation:
# ENERGY :
shift_ = 18000.0 # e.g. mean , lookup from stats, you can round it up
scale_ = 40000.0 # e.g. 2xS.D., lookup from stats
# x_norm = (x-shift_)/scale_

# WIND SPEED:
wind_scale_ = 8.0 # half of max speed (wind vector)

# location of networks
model_dir = os.path.relpath('./best_models')
# get all models that match the pattern:
model_filenames = glob.glob(os.path.join(model_dir,'24Jul2020_*'))

# Just in case
deployment_end_time  = pd.to_datetime('2020-12-29 10:00:00')

# # # # # # # # # # # # # # # # # # #
#        HELPER FUNCTIONS           #
# # # # # # # # # # # # # # # # # # #
# my server's offset time:
time_offset = pd.Timedelta(minutes=0,seconds=0) # correction to cpu time
# jhub offset--> pd.Timedelta(minutes=6,seconds=-9)
if TESTING:
    time_offset = pd.Timedelta(hours=10,minutes=0,seconds=0)

def utc_now(time_delta = time_offset):
    t = datetime.datetime.utcnow()
    utc_time = pd.to_datetime('{}-{:02d}-{:02d} {:02d}:{:02d}:{:02d}'.format(
        t.year,t.month,t.day, t.hour,t.minute,t.second))+time_delta
    return utc_time

def need_time():
    '''Current time needed for Energy kWh readings'''
    curr_time_utc = utc_now()
    curr_minutes = curr_time_utc.minute
    
    if (curr_minutes<10) or (curr_minutes>29):
        return curr_time_utc.round('H')-pd.Timedelta(hours=1)
    return curr_time_utc.round('H')

def sleep4time(curr_utc_time):
    # sleep until 50min
    min_now = curr_utc_time.minute + curr_utc_time.second/60
    if (min_now<50) and (min_now>10):
        # deploy time 1: wait for HH:51min
        time_left = (50-min_now)*60 # in seconds
    elif (min_now>=50):
        # deploy time 2: wait for HH:00min
        time_left = (60-min_now)*60 # in seconds
    elif min_now<9:
        #deploy time 3: wait for HH:09min
        time_left = (9-min_now)*60 # in seconds
    elif (min_now>9.5) and (min_now<50):
        # deploy time 1: wait for HH:51min
        time_left = (50-min_now)*60 # in seconds
    if (curr_utc_time.minute==0) and (curr_utc_time.second<15):
        time_left = 0.0
    print(f'\nSleep {time_left/60:2.0f}mins\n')
    time.sleep(abs(time_left))

def send_value2url(val):
    html=b''
    passwrd = '961993551'
    url_add = f'http://3.1.52.222/submit/pred?pwd={passwrd}&value={val}'
    try:
        with urllib.request.urlopen(url_add) as response:
            html = response.read()
    except urllib.request.HTTPError:
        html = b'server is down'
        print(utc_now(),f' (for {need_time()}):',html.decode(),url_add)
    return html


def read_wind_forecasts_w_range(wind_speed_range):
    dfs = []
    for model_n in range(1,3,1):
        for farm_k in range(len(datautils.locations)):
            dfs.append(
                datautils.readlocation_as_vec( model_n,farm_k, wind_speed_range).interpolate(
                    method='linear').reindex(wind_speed_range) )
    # concatenate along axis 1 using datetime as reference
    # then take average of two forecasts
    return datautils.average_forecast_models( pd.concat(dfs,axis=1) )


modellist = []
for fname in model_filenames:
    with open(fname, 'rb') as f:
        print(f'Loading {fname}')
        modellist.append(pickle.load(f))

for net in modellist:
    net.to(device)

def predictT18(x,y_0,nets=modellist):
    results = []
    y_0 = torch.from_numpy(y_0.astype(np.float32)).to(device)
    x = torch.from_numpy(x.astype(np.float32)).to(device)
    with torch.no_grad():
        for net in modellist:
            net.eval()
            results.append(net(x,y_0).detach().cpu().numpy())
    return results



def iter_predict18(energy_df, latest_energy_time_):
    tau_fill = pd.date_range(latest_energy_time_+pd.Timedelta(hours=1), need_time(), freq='H')
    
    if tau_fill.shape[0]>39:
        # if missing more than 39 hours just return 0
        print(f'\nToo many missing values for energy (missing {tau_fill.shape[0]} recent samples.)\n')
        return 0
    # time range for Energy
    eng_fill_range = pd.date_range(
        tau_fill[0] - pd.Timedelta(hours=window_size-1+18), energy_df.index[-1], freq='H')
    # normalised energy (copy)
    eng_df_norm = ( energy_df.loc[eng_fill_range].copy(deep=True)- shift_)/scale_
    
    # time range for wind speed
    wind_speed_range = pd.date_range(eng_fill_range[0],
                                     need_time()+pd.Timedelta(hours=18),freq='H')
    wind_df_norm = read_wind_forecasts_w_range(wind_speed_range)/wind_scale_
    
    X_clock = np.cos(np.pi*(tau_fill.hour.values)/23)**2
    
    # fill in missing values
    for k, t in enumerate(tau_fill):
        need_range = pd.date_range(t-pd.Timedelta(hours=window_size-1+18),
                                   t-pd.Timedelta(hours=18),freq='H')
        Y_fill = eng_df_norm.loc[need_range].values.reshape(1,-1)
        Y0_fill = Y_fill[:,:1]
        Ydiff_fill = Y_fill[:,0:-1] - Y_fill[:,1:]
        # wind needs T+18 forecast
        need_range_wind = pd.date_range(need_range[0],
                                        need_range[-1]+pd.Timedelta(hours=18),freq='H')
        
        X_df = wind_df_norm.loc[need_range_wind].values
        X_norm = np.sqrt(X_df[:,0::2]**2 + X_df[:,1::2]**2)# wind speed from wind vectors
        X_windows = []
        for l in range(X_norm.shape[1]):
            X_windows.append(
                datautils.windowed_diff_data(X_norm[:,l], lead_time=lead_time, window_size=window_size))
        
        X_fill = [Ydiff_fill]
        X_fill.extend(X_windows)
        X_fill.append(X_clock[k].reshape(1,1))
        X_fill = np.concatenate(X_fill, axis=1)
        eng_df_norm.loc[t] = np.mean(predictT18(X_fill,Y0_fill))
    
    # Predict T+18
    # Y(T+0) and diff-s
    need_range = pd.date_range(tau_fill[-1]-pd.Timedelta(hours=window_size-1),tau_fill[-1],freq='H')
    Y = eng_df_norm.loc[need_range].values.reshape(1,-1)
    Y0 = Y[:,:1]
    Ydiff = Y[:,0:-1] - Y[:,1:]
    
    # X(T+18), X(T+0), and diff-s 
    # wind needs T+18 forecast
    need_range_wind = pd.date_range(need_range[0], need_range[-1]+pd.Timedelta(hours=18),freq='H')
    X_df = wind_df_norm.loc[need_range_wind].values
    X_norm = np.sqrt(X_df[:,0::2]**2 + X_df[:,1::2]**2)# wind speed from wind vectors
    
    # Time of day T+18
    X_clock = np.cos(np.pi*(need_time()+pd.Timedelta(hours=18)).hour/23)**2
    X_clock = X_clock.reshape(-1,1)
    
    X_windows = []
    for l in range(X_norm.shape[1]):
        X_windows.append(
            datautils.windowed_diff_data(X_norm[:,l], lead_time=lead_time, window_size=window_size))
    
    Xdeploy = [Ydiff]
    Xdeploy.extend(X_windows)
    Xdeploy.append(X_clock)
    Xdeploy = np.concatenate(Xdeploy, axis=1)
    
    # Predict using 5 best models, and de-normalise, take mean for all predictions:
    Y_pred = np.mean(np.array(predictT18(Xdeploy,Y0))*scale_ +shift_)
    # Set 0kWh as min prediction, and convert to integer
    Y_pred = np.maximum(0,int(Y_pred))
    
    return Y_pred

Available device: cuda:3
Loading best_models/24Jul2020_0150_ver5.pkl
Loading best_models/24Jul2020_0150_ver4.pkl
Loading best_models/24Jul2020_0150_ver2.pkl
Loading best_models/24Jul2020_0150_ver1.pkl
Loading best_models/24Jul2020_0150_ver3.pkl


In [149]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
print('\n'+'- - '*5+
      f'\nDeployment > {"Testing" if TESTING else " ON  <"}\n'+
      '- - '*5+'\n')
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # DEPLOYMENT LOOP # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
while deployment_end_time>utc_now():
    if not TESTING:
        sleep4time(utc_now())
        t0 = utc_now()
        # Download New Forecasts:
        datautils.download_forecasts_all()
        # AI4Impact source:
        datautils.download_energy_latest()
        print(f'---\nTime elapsed: {utc_now()-t0}\n---\n')
    
    # energy readings
    energy_df = datautils.read_ai4impact_energy('datasets/energy-ile-de-france.csv')
    energy_date_range = pd.date_range(energy_df.index[0],energy_df.index[-1],freq='H') 
    
    need_range = pd.date_range(need_time()-pd.Timedelta(hours=window_size-1),need_time(),freq='H')
    
    if energy_df[energy_df.index==need_time()].values.shape[0]==0:
        print(f'\nUsing iterative method: latest {energy_date_range[-1]} (need {need_time()})\n')
        Y_pred = iter_predict18(energy_df, energy_date_range[-1])
        pred_method = f'iterative ({energy_date_range[-1]})'
    else:
        print(f'\nComputing T+18 forecast directly (latest {energy_date_range[-1]}, need {need_time()})\n')
        # we need this range of dates for features
        oldest_time = need_time()-pd.Timedelta(hours=window_size-1) # past data with window_size
        wind_speed_latest_time = need_time()+pd.Timedelta(hours=18) # T+18 wind
        wind_speed_range = pd.date_range(oldest_time,wind_speed_latest_time,freq='H') # select window_size and all the way to T+18
        
        wind_df = read_wind_forecasts_w_range(wind_speed_range)
        wind_norm = wind_df.values/wind_scale_
        wind_speeds = np.sqrt(wind_norm[:,0::2]**2 + wind_norm[:,1::2]**2)# wind speed from wind vectors
        
        # Time of day T+18
        X_clock = np.cos(np.pi*(need_time()+pd.Timedelta(hours=18)).hour/23)**2
        X_clock = X_clock.reshape(-1,1)
        
        X_norm_diffwindows = []
        for l in range(wind_speeds.shape[1]):
            X_norm_diffwindows.append(
                datautils.windowed_diff_data(wind_speeds[:,l], lead_time=lead_time, window_size=window_size))
        # print for debugging
        # print('\nX(t+18),X(T+0),X(T+0)-X(T-1),...,X(T-window_size+2)-X(T-window_size+1):\n',
        #       [l.shape for l in X_norm_diffwindows])
        Y = (energy_df.loc[need_range].values[::-1,0].reshape(1,-1) - shift_)/scale_
        Y0 = Y[:,:1]
        Ydiff = Y[:,0:-1] - Y[:,1:]
        Xdeploy = [Ydiff]
        Xdeploy.extend(X_norm_diffwindows)
        Xdeploy.append(X_clock)
        Xdeploy = np.concatenate(Xdeploy, axis=1)
        # Predict using 5 best models, and de-normalise, take mean for all predictions:
        Y_pred = np.mean(np.array(predictT18(Xdeploy,Y0))*scale_ +shift_)
        # Set 0kWh as min prediction, and convert to integer
        Y_pred = np.maximum(0,int(Y_pred))
        pred_method = f'direct ({energy_date_range[-1]})'
        
    curr_utc_time = utc_now()
    curr_mins = curr_utc_time.minute +curr_utc_time.second/60
    if (curr_mins<50) and (curr_mins>10):
        continue
    url_response = send_value2url(Y_pred)
    print(url_response.decode())
    with open('rt_predictions.txt', 'a') as the_file:
        the_file.write(f'{utc_now()}: {Y_pred} (for {need_time()}, {pred_method}) {url_response.decode()}\n')
    print(f'---\n {utc_now()}: Time elapsed: {utc_now()-t0}\n---\n')



- - - - - - - - - - 
Deployment >  ON  <
- - - - - - - - - - 


Sleep  7mins



KeyboardInterrupt: 

In [152]:
!ps -aux |grep deploy

murat    14113  0.0  0.0 131828  1240 pts/1    S+   03:49   0:00 screen -r my_deploy
murat    14803  6.6  0.3 29290976 1601588 pts/6 Sl+ 03:56   0:04 python deploy_models_v2.py
murat    14923  0.0  0.0 113196  1408 pts/8    Ss+  03:57   0:00 /usr/bin/sh -c ps -aux |grep deploy
murat    14925  0.0  0.0 112700   756 pts/8    S+   03:57   0:00 grep deploy
murat    23889  0.0  0.0 132124  1696 ?        Ss   Jul21   0:00 SCREEN -S my_deploy
